# Camada Bronze

In [0]:
CREATE TABLE taxi_trip_bronze.tax_trip_table
USING CSV
OPTIONS (
  path '/Volumes/workspace/taxi_trip_bronze/landing_zone/yellow_tripdata_2015-01.csv,
  /Volumes/workspace/taxi_trip_bronze/landing_zone/yellow_tripdata_2016-01.csv,
  /Volumes/workspace/taxi_trip_bronze/landing_zone/yellow_tripdata_2016-02.csv,
  /Volumes/workspace/taxi_trip_bronze/landing_zone/yellow_tripdata_2016-03.csv',
  header 'true',
  inferSchema 'true'
);